# Word2Vecを用いた類語の取得
<!--
[学習済みのword2vecのモデル](https://github.com/Kyubyong/wordvectors)

[学習済みのword2vecのモデルのロード](https://blog.amedama.jp/entry/gensim-fasttext-pre-trained-word-vectors)
-->
[データ](https://fasttext.cc/docs/en/crawl-vectors.html)

In [ ]:
#!curl https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz --output cc.ja.300.vec.gz 

### 出力用のディレクトリを作成する

In [1]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [2]:
data_path = '/workspaces/ycu-gradiation/DataSet/wiki-news-300d-1M.vec'
type(load_vectors(data_path))


### 読み込みに時間がかかる(10分程度)

- pickle でダンプしておく

In [2]:
%%time
import gensim
binary=True
# data_path='/workspaces/ycu-gradiation/DataSet/cc.en.300.vec.gz'; binary=False 
# data_path='/workspaces/ycu-gradiation/DataSet/wiki.en.vec' ; binary=False  
# data_path='/workspaces/ycu-gradiation/wiki.en.align.vec' ; binary=False  
data_path='/workspaces/ycu-gradiation/wiki-news-300d-1M.vec' ; binary=False  

model = gensim.models.KeyedVectors.load_word2vec_format(data_path, binary=binary,
                                                        unicode_errors='ignore')

import pickle as pkl
out_path=os.path.join(data_path+'.pkl')
with open(out_path,'wb') as fw:
    pkl.dump(model,fw)

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/ycu-gradiation/wiki-news-300d-1M.vec'

# 事前保存のデータを読み込み(直接読むより早い)

In [3]:
%%time
import pickle as pkl
# data_path='wiki.en.vec'
# data_path='cc.en.300.vec.gz'
# data_path='wiki.en.align.vec'
data_path='wiki-news-300d-1M.vec'
# data_path='wiki.en.vec'
out_dir = "/workspaces/test/ycu-api/DataSet"
in_path=os.path.join(out_dir,data_path+'.pkl')
with open(in_path,'rb') as fr:
    model0=pkl.load(fr)

CPU times: user 1.78 s, sys: 3.02 s, total: 4.8 s
Wall time: 4.84 s


In [4]:
type(model0)

gensim.models.keyedvectors.KeyedVectors

In [5]:
print(list(model0.index_to_key)[:50])
print(len(model0.index_to_key))


[',', 'the', '.', 'and', 'of', 'to', 'in', 'a', '"', ':', ')', 'that', '(', 'is', 'for', 'on', '*', 'with', 'as', 'it', 'The', 'or', 'was', "'", "'s", 'by', 'from', 'at', 'I', 'this', 'you', '/', 'are', '=', 'not', '-', 'have', '?', 'be', 'which', ';', 'all', 'his', 'has', 'one', 'their', 'about', 'but', 'an', '|']
999994


In [14]:
#model0.key_to_index

In [33]:
model0['cat'].shape

(300,)

In [4]:
en_word_list = pd.read_csv('/workspaces/test/ycu-api/DataSet/english_words_list.csv',usecols=['Lemma\n（見出し語）','Rank\n(順位)','Frequency\n(頻度)','Commentary\n(解説)']).sort_values('Frequency\n(頻度)',ascending=False)
# en_word_list.dropna().info()
print(en_word_list.head())

NameError: name 'pd' is not defined

In [5]:
from difflib import SequenceMatcher
import pandas as pd

en_word_list = pd.read_csv('/workspaces/test/ycu-api/DataSet/english_words_list.csv',usecols=['Lemma\n（見出し語）','Rank\n(順位)','Frequency\n(頻度)','Commentary\n(解説)']).sort_values('Frequency\n(頻度)',ascending=False)


def get_synonyms_w2v(text,model):
    results = []
    #for word, sim in model.wv.most_similar(text, topn=10):
    for word, sim in model.most_similar(text, topn=20):
        results.append({'term': word, 'similarity': sim})
    return results

def get_thema_w2v(text,model):
    results = []
    return_list = []
    #for word, sim in model.wv.most_similar(text, topn=10):
    thema_len = len(text)

    for word, sim in model.most_similar(text, topn=100):        
        word_len = len(word)
        if thema_len > word_len:
             r = max([SequenceMatcher(None, word, text[i:i+word_len]).ratio() for i in range(thema_len-word_len+1)])
             if r >0.8:
                 print(r,word)
                 pass
             else:
                 results.append({'term': word, 'similarity': sim,'r':r})
                 return_list.append(word)
        elif thema_len > word_len:
             r = max([SequenceMatcher(None, text, word[i:i+thema_len]).ratio() for i in range(word_len-thema_len+1)])
             if r >0.8:
                 print(r,word)
                 pass
             else:
                 results.append({'term': word, 'similarity': sim,'r':r})
                 return_list.append(word)
        else:
            pass
    return return_list


def calc_similarity_w2v(text1, text2,model):
    sim = model.wv.similarity(text1, text2)
    return sim

def suggestion_thema(sentence_list,thema_list):
    suggestion_thema_list = list(set(thema_list) - set(sentence_list))
    print(len(suggestion_thema_list))
    return en_word_list[en_word_list['Lemma\n（見出し語）'].isin(suggestion_thema_list)].drop('Rank\n(順位)', axis=1)

# 英文の分解
def sentence_to_word(text):
    word_list = []
    tmp_lst = text.replace('?','').replace('!','').replace('.','').split()                     
    for word in tmp_lst:                       
        word_list.append(word)  
    return word_list

    




In [6]:
sentence = "I think that we should try to reduce trash and make the earth clean more. And usually we don't think of trash. But it is big prpblem to burn trash. Because when we burn it ,it relese CO2. So the earth is warmer and warmer. So we have to do something to reduce trash. We usually buy foods、cook and eat them. But we often throw away them that isn't eaten by us. I think that we stop this behavior. We should think amount of we can eat and buy foods So the earth will become clean and beautiful more."

In [7]:
thema_list = get_thema_w2v('environment',model0)
sentence_list = sentence_to_word(sentence)
thema = suggestion_thema(sentence_list,thema_list)


0.9 enviroment
0.9 evironment
0.875 environs
0.9 envionment
70


In [8]:
print(thema.values.tolist())

[['life', 12842.0, '生活. 一生. 人生. 生命. 命'], ['world', 12684.0, '世界'], ['system', 11725.0, 'システム. 組織. 仕組み'], ['area', 8441.0, '地域. エリア. 面積'], ['business', 7708.0, 'ビジネス. 仕事'], ['health', 5923.0, '健康. 健康状態. 調子'], ['culture', 4656.0, '文化'], ['community', 4162.0, '共同体. 生活共同体. 群生. 共同社会. 一般社会. 団体. 公衆'], ['technology', 3884.0, 'テクノロジー. 科学技術. 工芸学'], ['industry', 3469.0, '産業. 取り引き. 工業. 生産'], ['situation', 3021.0, '状態. 場所. 位置. 立場. 情勢'], ['education', 2877.0, '教育. 素養. 教育学'], ['economy', 2797.0, '経済. 経済組織. 節約'], ['society', 2510.0, '社会'], ['nature', 2085.0, '自然. 性質'], ['facility', 1759.0, 'たやすさ. 容易さ. 設備. 施設. 腕前'], ['organism', 1339.0, '有機体. 生物. 人間. 個々の小さな動･植物. 有機的組織体(社会など)'], ['context', 1297.0, '前後関係. 状況. 文脈. 背景'], ['attitude', 1070.0, '態度. 心構え. 姿勢'], ['manner', 1064.0, '方法. 態度. 行儀. 風習. 様式. 礼儀作法'], ['landscape', 736.0, 'の風景を美化する. 庭師をする. 景色. 風景画'], ['atmosphere', 636.0, '雰囲気. 環境. 空気. 気圧. 媒体ガス. 触媒. 大気'], ['pollution', 581.0, '公害. 汚すこと. 汚染. 汚れ'], ['planet', 545.0, '惑星'], ['climate', 519.0, '風土. 気候. 地方.

In [19]:
probe_words =  sentence_to_word(sentence)

for probe_word in probe_words:
    word_similar=get_synonyms_w2v(probe_word,model0)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{probe_word} ~ \n{words}\n')

He ~ 
['he', '.He', '7.He', 'She', 'aHe', 'etc.He', 'It', 'too.He', '-He', '6.He', 'His', '20.He', 'IHe', 'him', '2.He', '4.He', '9.He', 'it-he', '-he', 'think.He']

are ~ 
['were', 'aren', 'arer', 'ARE', 'arent', 'aare', "'re", 'They', 'sre', 'they', 'are--they', 'arere', 'may', 'iare', 'areing', 'will', 'have', 'arehe', 'wre', 'should']

moving ~ 
['move', 'Moving', 'relocating', 'moved', 'shifting', 'moves', 'moving.If', 'moving.I', 'moving-', 'moveing', 'movng', 'settling', 'moving.And', 're-locating', 'moving.We', 'moving.The', 'moving.', 'moving.It', 'MOVING', 'dragging']

here ~ 
['here.So', 'Here', 'here.But', 'here.Now', 'here.I', 'here.Well', 'here.As', 'here-', 'here.OK', 'here.That', 'here--', 'here.And', 'here.It', 'here.In', 'here.', 'here.If', 'here.We', 'here.Anyway', 'here.While', 'here.There']

I ~ 
['So', "'m", "'d", '.I', '-I', 'so', 'am', 'we', 'iI', 'i', 'OK--I', 'if', 'think', 'thI', 'do-I', 'Ok', 'tI', 'sI', 'wI', 'weI']

am ~ 
["'m", 'I', 'Im', 'I.m', 'm', 'amI

In [31]:
# probe_words=['cat','dog','NewYork',
#              #'自由の女神',
#              'children',
#              'school',
#              'govement',
#              #'6月',
#              #'テレビドラマ',
#              'comment',
#              'summer',
#              'WTO',
#              'SARS',
#             ]

# for probe_word in probe_words:
#     word_similar=get_synonyms_w2v(probe_word,model0)
#     words=[word_entry['term'] for word_entry in word_similar]
#     print(f'{probe_word} 〜　\n{words}\n')

    
    

cat 〜　
['cats', 'kitty', 'kitten', 'feline', 'moggie', 'cat.It', 'dog', 'cat.The', 'cat.I', 'moggy']

dog 〜　
['dogs', 'puppy', 'pup', 'canine', 'pet', 'doggie', 'dog--', 'beagle', 'dachshund', 'cat']

NewYork 〜　
['York', 'Yorkand', 'New-York', 'Newyork', 'York.', 'YorkCity', 'YOrk', 'York.I', 'York-', 'N.Y']

children 〜　
['kids', 'chidren', 'chidlren', 'chldren', 'parents', 'adults', 'chilren', 'childred', 'child', 'chiildren']

school 〜　
['schoo', 'schools', 'school.The', 'schoool', 'kindergarten', 'shcool', 'school.All', 'shool', 'school.And', 'school-']

govement 〜　
['governemt', 'govenment', 'goverenment', 'govenrment', 'governmetn', 'governement', 'governmet', 'goverment', 'govrnment', 'gvt']

comment 〜　
['comments', 'commment', 'commenting', 'comment.', 'comment.This', 'commments', 'comment.And', 're-comment', 'comment.That', 'post']

summer 〜　
['summertime', 'winter', 'summers', 'mid-summer', 'summmer', 'autumn', 'spring', 'summer.This', 'Summer', 'summer-']

WTO 〜　
['W.T.O.', '

# アナロジー　（意味の加減算)

In [18]:
def analogy(X_Y, x,model):
    X, Y = X_Y
    results = []
    #for word, sim in model.wv.most_similar(positive=[Y, x], negative=[X], topn=10):
    for word, sim in model.most_similar(positive=[Y, x], negative=[X], topn=10):
        results.append({'term': word, 'similarity': sim})
    return results

In [22]:
X_Y_x_list=[( 'Franch','Paris','Germany'),  #首都
            ('England','London','Japan'),  #首都
            ('boy','girl','husband'),  #性別
            ('Japan','Tokyo', 'Korea' ),  # NG 韓国がボキャブラリー辞書にない
            ('Japan','Tokyo', 'China'),   # NG　中国がボキャブラリー辞書にない
           ]

for X,Y, x in X_Y_x_list:
    word_similar=analogy((X,Y),x,model0)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{Y} - {X} + {x} 〜　\n{words}\n')



Paris - Franch + Germany 〜　
['Berlin', 'Munich', 'Hamburg', 'Frankfurt', 'Leipzig', 'Düsseldorf', 'Cologne', 'Stuttgart', 'Vienna', 'Bonn']

London - England + Japan 〜　
['Tokyo', 'Toyko', 'Osaka', 'Tokyo-', 'Fukuoka', 'Shinjuku', 'Nagoya', 'Tokyo.The', 'Yokohama', 'Shibuya']

girl - boy + husband 〜　
['wife', 'fiance', 'mother-in-law', 'sister-in-law', 'daughter', 'ex-husband', 'huband', 'step-daughter', 'fiancé', 'fiancee']

Tokyo - Japan + Korea 〜　
['Seoul', 'Pyongyang', 'Daegu', 'Gwangju', 'Busan', 'Seoul.The', 'Pyonyang', 'Daejeon', 'Kwangju', 'Gyeongsan']

Tokyo - Japan + China 〜　
['Beijing', 'Shanghai', 'Chongqing', 'Bejing', 'Nanjing', 'Shenzhen', 'Chengdu', 'Hangzhou', 'Guangzhou', 'Tianjin']



# 日本語Wikipedia 全ページの文章でWord2Vecを学習

In [ ]:
import re
import unicodedata
from bs4 import BeautifulSoup
#import lec05_parser as parser
import MeCab
from glob import glob
import os
from tqdm.notebook import tqdm

# クレンジング：正規化と空白の圧縮

In [ ]:
translation_table=str.maketrans(dict(zip('()!', '（）！')))  # 半角 -> 全角  
#
#  クレンジング：　正規化と，　空白の圧縮
#
def cleanse(text,debug=False):
    if debug:
        return text
    text=unicodedata.normalize('NFKC', text).translate(translation_table)  #  テキストの正規化
    text=re.sub(r'\s+', ' ',text)  # 空白の正規化 半角の空白1つに。
    #text=cleanse_post(text)
    return text

def insert_nl_at_eos_char(text):  # insert newline's
    text=re.sub('([。])', '\\1\n',text)
    return text

def cleanse_ext(text):  #  for word2vec special
    text=re.sub('Section::::','',text)
    text=re.sub('[！「」『』（），、]','',text)
    return text
                

# 文単位で分かち書きされたファイルの作成

In [ ]:
# コーパス作り
WIKIDUMP_DIR='./wikipedia_text'
eos_mark='<EOS>'

def get_each_contetns_by_text(wikidir):
    for file in tqdm(glob(os.path.join(wikidir,'*/wiki_*'))):
        #print(file)
        with open(file,'r') as f:
            read_text=f.read()
            soup=BeautifulSoup(read_text)
            for doc in soup.find_all(['doc']):
                text=''
                input_text=doc.text
                input_text=cleanse(input_text)
                input_text=insert_nl_at_eos_char(input_text)
                for line in input_text.split('\n'):
                    line=line.strip()
                    if len(line)>0:
                        #print(f'**{line}**')
                        text += cleanse_ext(line)
                        if line[-1] in ['。','！']:
                            text=text[:-1]+eos_mark  # remove ['。','！'] but insert eos
                        else:
                            text+=eos_mark
                            #
                            pass
                        text += '\n'
                title=doc['title']
                yield cleanse(text), cleanse(title)
                #yield text, title
    
def convert_and_concat_wakati(out_dir='.'):
    with open(os.path.join(out_dir,'wiki_wakati_nl_Q.txt'),'w') as fw,\
        open(os.path.join(out_dir,'wiki_title_Q.txt'),'w') as f_title:
        count=0    
        #mecab=parser.MeCab('-Owakati')
        mecab= MeCab.Tagger('-Owakati')
        for text, title in  get_each_contetns_by_text(WIKIDUMP_DIR):
            print('　'*60,end='\r')
            print(f'{count}:{title[:45]}',end='\r')
            f_title.write(f'{count}:{title}\n')
            #print(mecab.parse(text))
            wakati_words=mecab.parse(text)
            #wakati_words=wakati.parse(text)   # こっちは エッフェル塔 が分解される
            wakati_words=repair_eos(wakati_words)
            fw.write(wakati_words)
            #print(text)
            if count>1:
                #break
                pass
            count += 1
            #if count>5: break  # for debug
        print('done')
        del mecab
        
def repair_eos(text):
    text=re.sub('< EOS > ','\n', text)
    return text
    
# def repair_eos(text):
#     text = '<SOS> '+text
#     text=re.sub('< EOS >','<EOS>\n<SOS>', text)
#     text=re.sub('<SOS>$','',text)
#     #print(f'* {text} *')
#     return text
    

# 分かち書き処理の実行

In [ ]:
%%time 
convert_and_concat_wakati(out_dir)

#  実行の記録
2020/11/11 <br>
```
見出し数：1,179,168
分かち書き処理：
CPU times: user 20min 54s, sys: 3min 18s, total: 24min 12s
Wall time: 1h 18min 2s
```

2021/11/18<br>
```
done101:福生市立福生第五小学校　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
CPU times: user 21min 48s, sys: 1min 49s, total: 23min 38s
Wall time: 24min 26s
```

# gensimのword2vec で
# 日本語wikipedia全コンテンツを学習

In [ ]:
from gensim.models import word2vec
vector_size=100
vector_size=300
%time data = word2vec.Text8Corpus(os.path.join(out_dir,'wiki_wakati_nl_Q.txt'))
%time model =  word2vec.Word2Vec(data, vector_size=vector_size)  # 100次元で学習
#
out_fname=f"wiki_{vector_size}.model"
out_path=os.path.join(out_dir,out_fname)
print(f'saving {out_path} ... ')
%time model1.save(out_path)  
print("ok") 

In [ ]:

out_fname=f"wiki_{vector_size}.model"
out_path=os.path.join(out_dir,out_fname)
print(f'saving {out_path} ... ')
%time model1.save(out_path)  
print("ok") 

#  実行の記録
2020/11/09 <br>
```
CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 9.06 µs
CPU times: user 1h 34min 30s, sys: 1min 7s, total: 1h 35min 38s
Wall time: 36min 14s
CPU times: user 2.07 s, sys: 1.13 s, total: 3.2 s
Wall time: 3.9 s
ok
```


```
ファイルサイズ：
wiki_wakati_nl.txt                 3,147,077,302 バイト（ディスク上の3.15 GB）
wiki.model                         67,779,585 バイト（ディスク上の70.8 MB）
wiki.model.trainables.syn1neg.npy  373,465,328 バイト（ディスク上の376.9 MB）
wiki.model.wv.vectors.npy          373,465,328 バイト（ディスク上の375.7 MB）
```

2021.11.18<br>
```
vector_size=100
CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.9 µs
CPU times: user 1h 17min 40s, sys: 41.5 s, total: 1h 18min 21s
Wall time: 27min 34s
CPU times: user 210 ms, sys: 920 ms, total: 1.13 s
Wall time: 1.19 s
ok
```

生成ファイル<br>
```
15787032  11 18 14:19 wiki.model
177484928 11 18 14:19 wiki.model.syn1neg.npy
177484928 11 18 14:19 wiki.model.wv.vectors.npy
```

```
vector_size=300
CPU times: user 8 µs, sys: 20 µs, total: 28 µs
Wall time: 31 µs
CPU times: user 1h 48minCPU times: user 8 µs, sys: 20 µs, total: 28 µs
Wall time: 31 µs
CPU times: user 1h 48min
saving wiki_300.model ... 
CPU times: user 249 ms, sys: 2.2 s, total: 2.45 s
Wall time: 2.67 s
oksaving wiki_300.model ... 
CPU times: user 249 ms, sys: 2.2 s, total: 2.45 s
Wall time: 2.67 s
ok

```

# 学習した Word2Vec の利用 (モデルロード）

In [ ]:
%%time
from gensim.models import word2vec
import gensim
vector_size=100
vector_size=300

in_fname=f"wiki_{vector_size}.model"
in_path=os.path.join(out_dir,in_fname)
print(f'loading {in_path} ... ')
model1 = gensim.models.Word2Vec.load(in_path) 

In [ ]:
type(model1)

# 同義語

In [ ]:
probe_words=['フランス','東京','ニューヨーク',
             #'自由の女神','ウルトラマン',
             #'中学校',
             '政府',
             #'6月',
             #'テレビドラマ',
             'ピカソ','夏','WTO','SARS','コロナ']
for probe_word in probe_words:
    word_similar=get_synonyms_w2v(probe_word,model1.wv)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{probe_word} 〜　\n{words}\n')

    

# アナロジー　（意味の加減算)

In [ ]:
vector_size=100
in_fname=f"wiki_{vector_size}.model"
in_path=os.path.join(out_dir,in_fname)
print(f'loading {in_path} ... ')
model100 = gensim.models.Word2Vec.load(in_path) 

In [ ]:
X_Y_x_list=[( 'フランス','パリ','ドイツ'),  #首都
            ('イギリス','ロンドン','日本'),  #首都
            ('日本','東京','ドイツ'),  #首都
            ('少年','少女','夫'),  #性別
            ('日本','東京', '韓国' ),  # NG 韓国がボキャブラリー辞書にない
            ('日本','東京', '中国'),   # NG　中国がボキャブラリー辞書にない
            #('パリ', 'エッフェル塔','東京'), # 観光資源
           ]

for X,Y, x in X_Y_x_list:
    print('model_300')
    print('*'*30)
    word_similar=analogy((X,Y),x,model1.wv)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{Y} - {X} + {x} 〜　\n{words}\n')
    #----
    print('model_100')
    word_similar=analogy((X,Y),x,model100.wv)
    words=[word_entry['term'] for word_entry in word_similar]
    print(f'{Y} - {X} + {x} 〜　\n{words}\n')




# ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ <br>

# ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ <br>


# ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ ★ <br>


